# [Module 1.1] 로컬 스크래치 훈련 (SageMaker 사용 안함)

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 기본 환경 세팅

## 참고:
- 세이지 메이커로 파이토치 사용 --> [Use PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)

---

# 1. 환경 셋업

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
import os
import time
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter

import pandas as pd
from IPython.display import display as dp

### 커스텀 라이브러리
import config 
import model 
import evaluate 
import data_utils 

# import src.model as model
# import src.evaluate as evaluate
# import src.data_utils as data_utils

# 2. 데이터 확인

## 2.1. Raw 파일 확인
- config.py 파일 확인 요망

In [4]:
! head -n5 {config.test_rating}

0	25	5	978824351
1	133	3	978300174
2	207	4	978298504
3	208	4	978294282
4	222	2	978246585


In [5]:
! head -n1 {config.test_negative}

(0,25)	1064	174	2791	3373	269	2678	1902	3641	1216	915	3672	2803	2344	986	3217	2824	2598	464	2340	1952	1855	1353	1547	3487	3293	1541	2414	2728	340	1421	1963	2545	972	487	3463	2727	1135	3135	128	175	2423	1974	2515	3278	3079	1527	2182	1018	2800	1830	1539	617	247	3448	1699	1420	2487	198	811	1010	1423	2840	1770	881	1913	1803	1734	3326	1617	224	3352	1869	1182	1331	336	2517	1721	3512	3656	273	1026	1991	2190	998	3386	3369	185	2822	864	2854	3067	58	2551	2333	2688	3703	1300	1924	3118


## 2.2. 훈련 및 검증용 데이터로 확인

In [7]:
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)




In [8]:
import numpy as np
print("train sahpe: ", np.asarray(train_data).shape)
print("test sahpe: ", np.asarray(test_data).shape)
print(f"user_num: {user_num}, item_num: {item_num}")
print(train_data[0:5])


train sahpe:  (994169, 2)
test sahpe:  (604000, 2)
user_num: 6040, item_num: 3706
[[0, 32], [0, 34], [0, 4], [0, 35], [0, 30]]


### 훈련 데이터

In [9]:
train_data_df = pd.DataFrame(train_data, columns=['user','item'])
print("train_df shape: ", train_data_df.shape)
print("train_df info: ", train_data_df.nunique())
train_data_df.sort_values(by=['user','item']).head()

train_df shape:  (994169, 2)
train_df info:  user    6040
item    3704
dtype: int64


,user,item
42,0,0
21,0,1
26,0,2
45,0,3
2,0,4


In [10]:
train_data_df.groupby('user').count().head()

,item
user,
0,52
1,128
2,50
3,20
4,197


### 검증 데이터

In [11]:
test_data_df = pd.DataFrame(test_data, columns=['user','item'])
dp(test_data_df.head())
dp(test_data_df.groupby('user').count().head())

,user,item
0,0,25
1,0,1064
2,0,174
3,0,2791
4,0,3373


,item
user,
0,100
1,100
2,100
3,100
4,100


# 3. 모델 훈련

## 3.1. 파라미터 설정

## 모델의 하이퍼파라미터 정의
- 하아퍼 파라미터 오브젝트 이름을 args 로 생성
    - 추후 SageMaker의 Script Mode 사용사에 args 오브젝트가 사용되기에, 이름을 맞추기 위해서 같은 이름을 사용 함
- 아래 파라미터는 로직 확인 용이기에, 훈련이 빨리 끝나기 위한 파라미터 값을 설정 함(에; learning rate)    

In [12]:
class Params:
    def __init__(self):
        self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        self.lr = 0.001
        self.top_k = 10
        self.out = True
        self.gpu = "0"
        

                
args = Params()
print("# of epochs: ", args.epochs)

# of epochs:  1


## 3.2. 데이터 셋 및 데이터 로더 정의

In [13]:
# construct the train and test datasets
train_dataset = data_utils.NCFData(
		train_data, item_num, train_mat, args.num_ng, True)
test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)
train_loader = data.DataLoader(train_dataset,
		batch_size=args.batch_size, shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



## 3.3. 모델 네트워크 생성

In [14]:
########################### CREATE MODEL #################################
if config.model == 'NeuMF-pre':
	assert os.path.exists(config.GMF_model_path), 'lack of GMF model'
	assert os.path.exists(config.MLP_model_path), 'lack of MLP model'
	GMF_model = torch.load(config.GMF_model_path)
	MLP_model = torch.load(config.MLP_model_path)
else:
	GMF_model = None
	MLP_model = None

NCF_model = model.NCF(user_num, item_num, args.factor_num, args.num_layers, 
						args.dropout, config.model, GMF_model, MLP_model)
NCF_model.cuda()



NCF(
  (embed_user_GMF): Embedding(6040, 32)
  (embed_item_GMF): Embedding(3706, 32)
  (embed_user_MLP): Embedding(6040, 128)
  (embed_item_MLP): Embedding(3706, 128)
  (MLP_layers): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.0, inplace=False)
    (7): Linear(in_features=64, out_features=32, bias=True)
    (8): ReLU()
  )
  (predict_layer): Linear(in_features=64, out_features=1, bias=True)
)

## 3.4. 손실 함수 및 옵티마이저 정의

In [15]:
loss_function = nn.BCEWithLogitsLoss()

if config.model == 'NeuMF-pre':
	optimizer = optim.SGD(NCF_model.parameters(), lr=args.lr)
else:
	optimizer = optim.Adam(NCF_model.parameters(), lr=args.lr)



## 3.5 훈련 루프 실행

In [16]:
print("=====> Staring Traiing <===========")
count, best_hr = 0, 0
for epoch in range(args.epochs):
    NCF_model.train() # Enable dropout (if have).
    start_time = time.time()
    train_loader.dataset.ng_sample()

    for user, item, label in train_loader:
        user = user.cuda()
        item = item.cuda()
        label = label.float().cuda()

        NCF_model.zero_grad()
        prediction = NCF_model(user, item)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        # writer.add_scalar('data/loss', loss.item(), count)
        count += 1

    NCF_model.eval()
    HR, NDCG = evaluate.metrics(NCF_model, test_loader, args.top_k)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
            time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
        if args.out:
            if not os.path.exists(config.model_path):
                os.mkdir(config.model_path)
            torch.save(NCF_model.state_dict(),'{}{}.pth'.format(config.model_path, config.model))

            

print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
									best_epoch, best_hr, best_ndcg))



=====> Staring Traiing <===========
The time elapse of epoch 000 is: 00: 02: 10
HR: 0.626	NDCG: 0.364
End. Best epoch 000: HR = 0.626, NDCG = 0.364


# 4. 로컬 추론

In [15]:
from evaluate import predict

In [16]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}

predict(NCF_model, payload, top_k=10)

[174, 128, 25, 273, 175, 1331, 1064, 464, 1963, 1902]

# 5. 훈련 스크립트로 실행

## 로컬에서 스크래치 훈련
- source/train_lib.py 에는 train(args) 함수가 정의 되어 있습니다.
- 이 함수에 인자를 넘기고 훈련을 합니다.
- 중요한 단계를 로깅 합니다.
    - 훈련 환경 셋업, 
    - 데이터 준비 및 데이터 로더 생성
    - 모델 네트워크 로딩
    - 모델 훈련 시작
    - 모델 훈련 완료
    - 모델 아티펙트 저장

In [48]:
class ParamsScript:
    def __init__(self):
        self.epochs = 1   
        self.lr = 0.1 # 0.001 오리지널 버전        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        self.top_k = 10
        self.out = True
        self.gpu = "0"
        self.model_dir = f"{config.model_path}"                                       
        self.train_data_dir = f"{config.main_path}"               
        self.test_data_dir = f"{config.main_path}"                       

                        
script_args = ParamsScript()
print("# of epochs: ", script_args.epochs)

# of epochs:  1


In [49]:
from train_lib import train

In [50]:
%%time 

train(script_args)

args.train_data_dir:  ../NCF-Data/
args.test_data_dir:  ../NCF-Data/
args.model_dir:  ./models/
=====> data loading <===========
=====> create data loader <===========
=====> Staring Traiing <===========
The time elapse of epoch 000 is: 00: 02: 10
HR: 0.130	NDCG: 0.064
End. Best epoch 000: HR = 0.130, NDCG = 0.064
CPU times: user 2min 14s, sys: 9.51 s, total: 2min 24s
Wall time: 2min 26s


# 6. 훈련된 가중치 로딩하여 추론

## 6.1. 추론시 사용할 모델 네트워크 설정 저장

In [20]:
# import json
# from src.common_utils import save_json, load_json

# model_config_dict = {
#     'user_num': str(user_num),
#     'item_num': str(item_num),
#     'factor_num' : str(args.factor_num),
#     'num_layers' : str(args.num_layers),
#     'dropout' : str(args.dropout),
#     'model_type': config.model
# }

# model_config_file = 'model_config.json'
# model_config_file_path = os.path.join(config.model_path, model_config_file)

# save_json(model_config_file_path, model_config_dict)
# # model_config_dict = load_json(model_config_file_path)    
# # model_config_dict

## 6.2. 모델 가중치 로딩하여 추론

In [21]:
# from inference import model_fn

In [22]:
# inf_model = model_fn(config.model_path)

In [23]:
# predict(inf_model, payload, top_k=10)